**https://github.com/tuhinjubcse/SarcasmGeneration-ACL2020**

#**Git Clone**

In [1]:
!git clone https://github.com/tuhinjubcse/SarcasmGeneration-ACL2020.git

Cloning into 'SarcasmGeneration-ACL2020'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (424/424), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 424 (delta 202), reused 279 (delta 100), pack-reused 0
Receiving objects: 100% (424/424), 309.86 KiB | 17.21 MiB/s, done.
Resolving deltas: 100% (202/202), done.


#**To just reverse the valence of the input (REVERSE)**

In [2]:
#!pip install -U PyYAML
import yaml
yaml.__version__

'6.0'

In [3]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import yaml

path = '/content/SarcasmGeneration-ACL2020/comet-commonsense/config/config.yaml'

def loadConfigForROV():
	with open(path) as f:
		docs = yaml.load_all(f, Loader=yaml.FullLoader)
		for doc in docs:
			for k, v in doc.items():
				if k=="exception_vadarneg_words":
					exception_vadarneg_words=v
				elif k=="missing_vadarneg_words":
					missing_vadarneg_words=v
	return exception_vadarneg_words,missing_vadarneg_words

def loadConfigForRank():
	with open(path) as f:
		docs = yaml.load_all(f, Loader=yaml.FullLoader)
		for doc in docs:
			for k, v in doc.items():
				if k=="swap":
					swap_words=v
	return swap_words

def loadConfigForSentences():
	with open(path) as f:
		docs = yaml.load_all(f, Loader=yaml.FullLoader)
		for doc in docs:
			for k, v in doc.items():
				if k=="nonoverlap":
					nonoverlap_words=v
	return nonoverlap_words




def loadConfigForRetrieval():
	with open(path) as f:
		docs = yaml.load_all(f, Loader=yaml.FullLoader)
		for doc in docs:
			for k, v in doc.items():
				if k=="stop_words":
					stop_words = v[0].split(',')
				if k=="missing":
					missing = v
				if k=="quantifier":
					quantifier = v
				if k=="replacement":
					replacements = v
				if k=="start":
					start = v
				if k=="wrongNE":
					wrongNE = v

	return stop_words,missing,quantifier,replacements,start,wrongNE


def loadConfig(step):
	if step == 'ROV':
		return loadConfigForROV()
	elif step == 'Retrieve':
		return loadConfigForRetrieval()
	elif step == 'Sentences':
		return loadConfigForSentences()
	else:
		return loadConfigForRank()

In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from loadconfig import loadConfig
import sys
import os


def getWordNetAntonyms():
	m= {}
	for line in open('/content/SarcasmGeneration-ACL2020/data/antonyms.txt'):
		m[line.strip().split()[0]]= line.strip().split()[1]
	return m


def findIfnegationPresent(utterance):
	words = utterance.split()
	for w in words:
		if w=='not' or w=='never' or  w=='Not' or w=='Never':
			return w,True
	return '',False

def findIfendingwithnt(utterance):

	d = {"didn't": "did","don't": "do","doesn't":"does","can't": "can",
	"cannot":"can","wouldn't":"would","shouldn't":"should"}
	words = utterance.split()
	for w in words:
		if w in d:
			return w,d[w],True
		if w.lower() in d:
			return w,d[w.lower()].capitalize(),True
	return '','',False


def getAntonym(word):
	antonyms = getWordNetAntonyms()
	if word.lower() not in antonyms:
		synonymsset = []
		antonymsset = []
		for syn in wn.synsets(word.lower()):
			for l in syn.lemmas():
				synonymsset.append(l.name())
				if l.antonyms():
					antonymsset.append(l.antonyms()[0].name())
		if len(antonymsset)==0:
			for w in synonymsset:
				if w in antonyms:
					return antonyms[w.lower()]
			return "not "+word
		else:
			return antonymsset[0]
	else:
		return antonyms[word.lower()]


def ifTwoNegation(utterance):
	exception_vadarneg_words, missing_vadarneg_words= loadConfig('ROV')
	utterance = utterance.replace(',','')
	sid = SentimentIntensityAnalyzer()
	arr = []
	sent = word_tokenize(utterance)
	for i in range(len(sent)):
		w = sent[i]
		if w == 'no':
			continue
		ss = sid.polarity_scores(w)
		if (ss['neg']==1.0 or w in missing_vadarneg_words) and (w not in exception_vadarneg_words):
			arr.append((w,i,abs(ss['compound'])))
	if len(arr)==2:
		if abs(arr[0][1]-arr[1][1])==2:
			return [arr[0][0],arr[1][0]],True
		else:
			return [arr[1][0]],True
	else:
		return [],False


def isThereOnlyOneNegation(utterance):
	exception_vadarneg_words, missing_vadarneg_words= loadConfig('ROV')
	sid = SentimentIntensityAnalyzer()
	count = 0
	word = ''
	arr = []
	for w in word_tokenize(utterance):
		if w=='no':
			continue
		ss = sid.polarity_scores(w)
		if ss['neg']==1.0 and w not in exception_vadarneg_words:
			count = count+1
			if count<=1:
				word = w
			arr.append(word)
		elif w in missing_vadarneg_words and count==0:
			count = count+1
			if count<=1:
				word = w
	if count==1:
		return word,True
	return 'cant_change',False



#TODO In future work used improved negation
#Current style/sentiment transfer techniques are still at low accuracy
def reverse_valence(utterance):
	#directly handle these without going for complicated logic
	utterance = utterance.lower()
	utterance = utterance.replace(' i ',' I ')
	if 'should be' in utterance or 'would be' in utterance:
		return utterance.replace(' be ',' not be ').capitalize()
	if ' need to ' in utterance:
		return utterance.replace(' need to ',' need not ').capitalize()
	if 'hate' in utterance:
		return utterance.replace('hate','love').capitalize()
	if 'least' in utterance:
		return utterance.replace('least','most').capitalize()
	if utterance.endswith('lies.'):
		return utterance.replace('lies','truth').capitalize()

	utterance = utterance.replace(" don't "," do not ")

	#check if negation present , in terms of single or double words or not/n't words
	word,verdict = findIfnegationPresent(utterance)
	negword,replneg,verdict1 = findIfendingwithnt(utterance)
	words,verdict3 = ifTwoNegation(utterance)
	negative, verdict2 = isThereOnlyOneNegation(utterance)

	#handle case by case , give priority to remove not first
	# print("here1",utterance)
	if verdict == True:
		return utterance.replace(word+' ','')
	elif verdict1==True and verdict2==False:
		return utterance.replace(negword,replneg)
	elif verdict3==True:
		for w in words:
			if getAntonym(w).startswith('not'):
				continue
			utterance = utterance.replace(w,getAntonym(w))
		return utterance
	else:
		prev_utterance = utterance
		utterance = utterance.replace(negative,getAntonym(negative))
		#incase algorithm could not handle still try to negate
		#cases replace present tense verbs by appending a don't
		#cases replace unique words prefixing with un 
		if utterance == prev_utterance:
			text = word_tokenize(utterance)
			pos_text = pos_tag(text)
			for a,b in pos_text:
				if b == 'VBP':
					utterance = utterance.replace(a,"don't "+a)
					break
				if a.startswith('un'):
					utterance = utterance.replace(a,a[2:])
					break
		utterance = utterance.split()
		for i in range(len(utterance)):
			if utterance[i] == 'an' and utterance[i+1][0] not in ['a','e','i','o','u']:
				utterance[i] = 'a'
		utterance = ' '.join(utterance)
		return utterance.capitalize()

#print(reverse_valence(sys.argv[1]))

In [7]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
print(reverse_valence(sys.argv[1]))

-f


In [9]:
print(reverse_valence('I love music'))

I don't love music


In [10]:
print(reverse_valence('I am happy'))

I don't am happy


In [11]:
print(reverse_valence('Today it will rain at 3 p.m'))

Today it will rain at 3 p.m


In [12]:
print(reverse_valence('I am sad'))

I am cheerful


In [13]:
print(reverse_valence('I will go to market at night'))

I will go to market at night


In [15]:
print(reverse_valence('I do not like to eat pizza much.'))

i do like to eat pizza much.
